In [7]:
get_ipython().system(u' pip install --upgrade pip')
get_ipython().system(u' pip install beautifulsoup4')
!pip install lxml
!pip install html5lib
!pip install requests

#Importing packages
import requests
from bs4 import BeautifulSoup

     |████████████████████████████████| 1.5MB 7.7MB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1


In [8]:
import pandas as pd

In [9]:
List_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(List_url).text

In [10]:
soup = BeautifulSoup(source, 'xml')

In [11]:
table=soup.find('table')

In [12]:
#Dataframe will consists of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [13]:
# Search all the postcode, borough, neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [14]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Data Cleaning

In [15]:
df=df[df['Borough']!='Not assigned']

In [16]:

df[df['Neighborhood']=='Not assigned']

df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [32]:
temp_df=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
temp_df=temp_df.reset_index(drop=False)
temp_df.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)

In [33]:
df_merge = pd.merge(df, temp_df, on='Postalcode')

In [34]:
df_merge.drop(['Neighborhood'],axis=1,inplace=True)

In [35]:
df_merge.drop_duplicates(inplace=True)

In [36]:
df_merge.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [37]:
df_merge.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [23]:

df_merge.shape

(103, 3)

# GEOCODER

In [1]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [3]:
import pandas as pd

In [4]:
geocoder_df=pd.read_csv('http://cocl.us/Geospatial_data')

In [5]:
geocoder_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
geocoder_df.rename(columns={'Postal Code':'Postalcode'},inplace=True)
geo_merged = pd.merge(geocoder_df, df_merge, on='Postalcode')

In [40]:

geocoder_data=geo_merged[['Postalcode','Borough','Neighborhood','Latitude','Longitude']]

In [42]:
geocoder_data.tail()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.706748,-79.594054
